In [1]:
import os, sys
import numpy as np
sys.path.append('../')

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
from onsstove.onsstove import OnSSTOVE
from onsstove.layer import RasterLayer, VectorLayer
from onsstove.raster import interpolate
import time

start = time.time()

## 1. Create an OnSSTOVE model

In [4]:
nepal = OnSSTOVE(project_crs=3857, cell_size=(1000, 1000))
output_directory = 'results'
nepal.output_directory = output_directory

## 2. Read the scenario data

In [5]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\OnSSTOVE cases\NP_test_file_social_specs.csv"
nepal.read_scenario_data(path, delimiter=',')

## 3. Read the cooking technologies data

In [6]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\OnSSTOVE cases\NP_test_file_tech_specs.csv"
nepal.read_tech_data(path, delimiter=',')

## 4. Add a country mask layer

In [7]:
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Administrative\npl_admbnda_nd_20201117_shp\npl_admbnda_adm0_nd_20201117.shp"
mask_layer = VectorLayer('admin', 'adm_0', layer_path=path)
os.makedirs(f'{output_directory}/admin/adm_0', exist_ok=True)
mask_layer.reproject(3857, f'{output_directory}/admin/adm_0')
nepal.mask_layer = mask_layer
# add admin_1 or 2 or 3 layer to gdf here

## 5. Add a population base layer

In [8]:
path = r"data\population_npl_2018-10-01.tif"
os.makedirs(f'{output_directory}/demographics/population', exist_ok=True)
population = RasterLayer('demographics', 'population', layer_path=path, resample='sum')
population.reproject(3857, f'{output_directory}/demographics/population', 1000, 1000)
population.mask(nepal.mask_layer.layer, f'{output_directory}/demographics/population')
nepal.base_layer = population
nepal.population_to_dataframe(population)

## 6. Calibrate population and urba/rural split

In [9]:
nepal.calibrate_pop()
nepal.calibrate_urban()

## 7. Calculate parameters of base fuel (Biomass)

### 7.1. Health costs

In [10]:
# Calculate relative risk for each disease for the base fuel
rr_0_alri, rr_0_copd, rr_0_ihd, rr_0_lc = nepal.base_fuel.relative_risk()

# Calculate the Population Atributable Factor (PAF) for each disease for the base fuel
paf_0_alri = nepal.base_fuel.paf(rr_0_alri, 1 - nepal.specs['clean_cooking_access'])
paf_0_copd = nepal.base_fuel.paf(rr_0_copd, 1 - nepal.specs['clean_cooking_access'])
paf_0_ihd  = nepal.base_fuel.paf(rr_0_ihd, 1 - nepal.specs['clean_cooking_access'])
paf_0_lc = nepal.base_fuel.paf(rr_0_lc, 1 - nepal.specs['clean_cooking_access'])

### 7.2. Carbon emissions and related costs

In [12]:
nepal.base_fuel.carb(nepal.specs, nepal.gdf)

### 7.3. Time for travelling and collecting fuel, and cooking

In [13]:
# paths to GIS layers
nepal.base_fuel.friction_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\2020_walking_only_friction_surface\2020_walking_only_friction_surface.geotiff"
nepal.base_fuel.forest_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\Forest cover\Global Forest Cover Change (GFCC).tif"

# Calculate total time
nepal.base_fuel.total_time(nepal.specs, population, output_directory)

# Add results to the model
nepal.raster_to_dataframe(nepal.base_fuel.total_time_yr, name='base_fuel_time', method='read')

# nepal.raster_to_dataframe(nepal.base_fuel.travel_time, name='base_fuel_travel_time', method='read')

## 8. Add wealth index GIS data

In [14]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Demand\Wealth Index\Wealth index 2011.tif"
wealth_index = RasterLayer('demographics', 'Wealth_index', layer_path=path,  resample='average')
os.makedirs(f'{output_directory}/demographics/Wealth_index', exist_ok=True)
interpolate(wealth_index.path)
wealth_index.align(population.path, f'{output_directory}/demographics/Wealth_index')

### 8.1. Calculate value of time

In [15]:
# Based on wealth index, minimum wage and a lower an upper range for cost of oportunity
nepal.get_value_of_time(wealth_index)

## 9. Read electricity network GIS layers

In [16]:
# Read MV lines
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\Nepal_DL0\Nepal_DL0.shp"
mv_lines = VectorLayer('electricity', 'MV_line', layer_path=path)
os.makedirs(f'{output_directory}/electricity/MV_line', exist_ok=True)
mv_lines.reproject(3857, f'{output_directory}/electricity/MV_line')

# Read HV lines
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\HV-network\HV_lines_georeferenced.shp"
hv_lines = VectorLayer('electricity', 'HV_line', layer_path=path)
os.makedirs(f'{output_directory}/electricity/HV_line', exist_ok=True)
hv_lines.reproject(3857, f'{output_directory}/electricity/HV_line')

### 9.1. Calculate distance to electricity infrastructure 

In [17]:
nepal.distance_to_electricity(hv_lines=hv_lines, mv_lines=mv_lines)

## 10. Add night time lights data

In [18]:
path = r"data\nighttime_lights.tif"
os.makedirs(f'{output_directory}/electricity/Night_lights', exist_ok=True)
ntl = RasterLayer('electricity', 'Night_lights', layer_path=path, resample='average')
ntl.align(population.path, f'{output_directory}/electricity/Night_lights')
ntl.layer[np.isnan(ntl.layer)] = 0 # hey lets make sure that we interpolate NaNs

nepal.raster_to_dataframe(ntl.layer, name='Night_lights', method='read')

## 11. Calibrate current electrified population

In [19]:
nepal.elec_current()

print('Calibrated electrified population fraction:', nepal.gdf['Elec_pop_calib'].sum() / nepal.gdf['Calibrated_pop'].sum())

We have identified the existence of transformers or MV lines as input data; therefore we proceed using those for the calibration
Calibrated electrified population fraction: 0.5201280449140765


## 12. Adding GIS data for LPG supply

In [20]:
nepal.techs['LPG'].lpg_path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\LPG\Nepal_Gas_12Jun2021_Final4.shp"
nepal.techs['LPG'].friction_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\2020_motorized_friction_surface\2020_motorized_friction_surface.geotiff"

nepal.techs['LPG'].add_travel_time(population, output_directory)

## 13. Calculate benefits and costs of each technology

In [21]:
names = ['Electricity', 'Collected_Traditional_Biomass', 'LPG']
techs = [nepal.techs[name] for name in names]

# Loop through each technology and calculate all benefits and costs
for tech in techs:
    print(f'Calculating health benefits for {tech.name}...')
    tech.morbidity(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)
    tech.mortality(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)
    print(f'Calculating carbon emissions benefits for {tech.name}...')
    tech.carbon_emissions(nepal.specs, nepal.gdf, nepal.base_fuel.carbon)
    print(f'Calculating time saved benefits for {tech.name}...')
    tech.time_saved(nepal.gdf, nepal.specs, population, output_directory)
    print(f'Calculating costs for {tech.name}...')
    tech.discounted_om(nepal.gdf, nepal.specs)
    tech.discounted_inv(nepal.gdf, nepal.specs)
    tech.discounted_meals(nepal.gdf, nepal.specs)
    tech.discount_fuel_cost(nepal.gdf, nepal.specs, nepal.rows, nepal.cols)
    tech.salvage(nepal.gdf, nepal.specs)
    print(f'Calculating net benefit for {tech.name}...\n')
    tech.net_benefit(nepal.gdf)

Calculating health benefits for electricity...
Calculating carbon emissions benefits for electricity...
Calculating time saved benefits for electricity...
Calculating costs for electricity...
Calculating net benefit for electricity...

Calculating health benefits for traditional_biomass_collected...
Calculating carbon emissions benefits for traditional_biomass_collected...
Calculating time saved benefits for traditional_biomass_collected...
Calculating costs for traditional_biomass_collected...
Calculating net benefit for traditional_biomass_collected...

Calculating health benefits for lpg...
Calculating carbon emissions benefits for lpg...
Calculating time saved benefits for lpg...
Calculating costs for lpg...
Calculating net benefit for lpg...



## 14. Getting the max benefit technology for each cell

In [22]:
nepal.maximum_net_benefit()

## 15. Printing the results

In [23]:
nepal.gdf

,geometry,Pop,Calibrated_pop,IsUrban,Households,base_fuel_time,value_of_time,HV_line_dist,MV_line_dist,Night_lights,...,benefits_electricity,net_benefit_electricity,costs_traditional_biomass_collected,benefits_traditional_biomass_collected,net_benefit_traditional_biomass_collected,costs_lpg,benefits_lpg,net_benefit_lpg,final_tech,maximum_net_benefit
0,POINT (9090104.530 3543192.836),31.171455,31.344224,0,5.498987,2993.471554,0.058780,58.872746,40.521599,0.000000,...,1519.404160,-99999.000000,0.916498,0.0,-0.916498,1471.833688,1454.987800,-16.845888,traditional_biomass_collected,-0.916498
1,POINT (9090104.530 3536192.836),46.757183,47.016337,0,8.248480,3442.932033,0.059025,52.392746,33.837849,0.000000,...,2503.428293,-99999.000000,1.374747,0.0,-1.374747,2190.039537,2406.803753,216.764215,lpg,216.764215
2,POINT (9083104.530 3535192.836),5.517799,5.548382,0,0.973400,2239.061114,0.059683,54.708317,35.846897,0.000000,...,227.520361,-99999.000000,0.162233,0.0,-0.162233,258.487731,216.117731,-42.369999,traditional_biomass_collected,-0.162233
3,POINT (9084104.530 3535192.836),15.585728,15.672112,0,2.749493,2280.672201,0.059637,54.203320,35.355339,0.000000,...,649.233158,-99999.000000,0.458249,0.0,-0.458249,729.396677,617.024977,-112.371700,traditional_biomass_collected,-0.458249
4,POINT (9064104.530 3534192.836),15.585728,15.672112,0,2.749493,2972.034660,0.064742,64.536812,47.634022,0.000000,...,800.473041,-99999.000000,0.458249,0.0,-0.458249,748.961096,768.264861,19.303765,lpg,19.303765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96717,POINT (9724104.530 3044192.836),98.039602,98.582991,0,17.295262,7652.475230,0.099102,13.038404,1.414214,0.000000,...,14661.035261,13007.293189,2.882544,0.0,-2.882544,NaN,14458.434699,NaN,electricity,13007.293189
96718,POINT (9797104.530 3044192.836),536.486856,539.460361,1,94.642169,3599.900735,0.101375,17.029387,0.000000,0.000000,...,42932.512093,33882.996678,15.773695,0.0,-15.773695,NaN,41823.852564,NaN,electricity,33882.996678
96719,POINT (9798104.530 3044192.836),315.580504,317.329624,0,55.671864,3660.375916,0.100924,17.117243,1.000000,0.000000,...,25510.636615,20187.392252,9.278644,0.0,-9.278644,NaN,24858.483951,NaN,electricity,20187.392252
96720,POINT (9722104.530 3043192.836),642.881959,646.445163,1,113.411432,7919.425593,0.104294,11.000000,0.000000,0.891932,...,103639.585887,92795.386950,18.901905,0.0,-18.901905,NaN,102311.059007,NaN,electricity,92795.386950


In [24]:
nepal.gdf.groupby(['Current_elec', 'final_tech']).agg({'Calibrated_pop': lambda row: sum(row) / 1000000,
                                       'maximum_net_benefit': lambda row: sum(row) / 1000000,
                                       'Pop': 'count'})

Calibrated_pop  \
Current_elec final_tech                                      
0            lpg                                  1.967741   
             traditional_biomass_collected        0.278837   
1            electricity                         26.363422   

                                            maximum_net_benefit    Pop  
Current_elec final_tech                                                 
0            lpg                                      17.043678  34305  
             traditional_biomass_collected            -0.008181   3810  
1            electricity                            1399.062391  58607

In [26]:
nepal.gdf.groupby(['Current_elec']).agg({'Calibrated_pop': lambda row: sum(row) / 1000000,
                                         'Elec_pop_calib': lambda row: np.nansum(row) / 1000000})

,Calibrated_pop,Elec_pop_calib
Current_elec,,
0,2.246578,0.000000
1,26.363422,14.880863


In [63]:
nepal.to_raster('final_tech')

Layer saved in results\Output\final_tech.tif

Variable codes:
    traditional_biomass_purchased: 0
    traditional_biomass_collected: 1
    improved_biomass_purchased: 2
    improved_biomass_collected: 3
    lpg: 4
    biogas: 5
    electricity: 6


In [29]:
end = time.time()

diff = end - start
print('Execution time:', str(str(int(diff//60))) + ' min ' + str(int((diff)%60)) + ' sec')

Execution time: 4 min 32 sec
